# Synchronize Object and Base Pose TF

This notebook synchronizes data from extracted bagfiles. Assuming the data directory containt are like so:
> ```
dataDir
    base.txt # index of pose of base by time
    object.txt # index of pose of object by time
```

The resulting output is like so:

> ```
dataDir
    base_sync.txt # index of pose of base by time
    object_sync.txt # index of pose of object by time
```

In [1]:
import errno
import glob
import numpy as np
import os
import pandas as pd
import shutil
from rgbd_benchmark_tools import associate

## Configure parameters

In [2]:
dataDir = '/home/ruffsl/Data/objects'
sample_rate = 1 # output sample rate (every x image pair)
frame_rate = 30 # input frame rate (fps)
tolerance = 0.6 # 60% period tolerance of frame rate
object_to_base_offset = 0 # in seconds

## Define helper functions

In [3]:
def insertionSort(alist):
    for index in range(1,len(alist)):
        currentvalue = alist[index]
        position = index
        while position>0 and alist[position-1][0]>currentvalue[0]:
            alist[position]=alist[position-1]
            position = position-1
        alist[position]=currentvalue
    return alist

## Create paths

In [4]:
base_file = os.path.join(dataDir,'base.txt')
object_file = os.path.join(dataDir,'object.txt')

base_sync_file = os.path.join(dataDir,'base_sync.txt')
object_sync_file = os.path.join(dataDir,'object_sync.txt')

## Load and associate data

In [5]:
base_list = associate.read_file_list(base_file)
object_list = associate.read_file_list(object_file)

max_difference = tolerance / frame_rate
object_base_matches = associate.associate(object_list, base_list, object_to_base_offset, max_difference)

## Sort data by time

In [6]:
object_base_matches = insertionSort(object_base_matches)

object_matches = np.array(object_base_matches)[:,0]
base_matches = np.array(object_base_matches)[:,1]

## Load pose info into dataframe

In [7]:
base_df = pd.read_csv(base_file, header=None, delim_whitespace=True, dtype={0:str})
base_df = base_df.set_index(0)

object_df = pd.read_csv(object_file, header=None, delim_whitespace=True, dtype={0:str})
object_df = object_df.set_index(0)

## Loop over matches and format as string for index

In [8]:
base_poses = []
object_poses = []
for object_pose, base_pose in object_base_matches[0::sample_rate]:
    base_poses.append("{:.6f}".format(base_pose))
    object_poses.append("{:.6f}".format(object_pose))

## Save pose file as well

In [9]:
base_df2 = base_df[base_df.index.map(lambda x: x in base_poses)]
base_df2.to_csv(base_sync_file, header=None)

object_df2 = object_df[object_df.index.map(lambda x: x in object_poses)]
object_df2.to_csv(object_sync_file, header=None)